In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d abhia1999/chronic-kidney-disease
!unzip \*.zip && rm *.zip

  0% 0.00/5.61k [00:00<?, ?B/s]
100% 5.61k/5.61k [00:00<00:00, 9.16MB/s]
Archive:  chronic-kidney-disease.zip
  inflating: new_model.csv           


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Step 1: Load Data
data = pd.read_csv('/content/new_model.csv')  # Replace '/path/to/your/dataset.csv' with the actual path

# Step 2: Prepare Data
X = data[['Bp', 'Sg', 'Al', 'Su', 'Rbc', 'Bu', 'Sc', 'Sod', 'Pot', 'Hemo']]
y = data['Class']  # Assuming 'Target' is the target variable name

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Define a function to calculate fuzzy membership values
def calculate_fuzzy_memberships(X_train, x_test, m):
    distances = np.linalg.norm(X_train - x_test, axis=1)
    max_distance = np.max(distances)
    fuzzy_memberships = np.exp(-((distances / max_distance) ** 2) / (2 * (m ** 2)))
    return fuzzy_memberships

# Step 6: Train and Evaluate the FKNN model
m = 2  # Fuzziness parameter
k = 5  # Number of neighbors
y_pred = []
for x_test in X_test_scaled:
    fuzzy_weights = calculate_fuzzy_memberships(X_train_scaled, x_test, m)
    knn_model = KNeighborsClassifier(n_neighbors=k, weights='distance')  # Weighted by fuzzy membership
    knn_model.fit(X_train_scaled, y_train, sample_weight=fuzzy_weights)
    y_pred.append(knn_model.predict([x_test])[0])

# Step 7: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Display evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Display classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


TypeError: KNeighborsClassifier.fit() got an unexpected keyword argument 'sample_weight'